# How to start

Before starting you must ensure that `scipp`, `mantid`, and `wfm_stitching` are on your `PYTHONPATH`.

For the first two follow instructions at: https://scipp.readthedocs.io/en/latest/getting-started/installation.html.
The last one can be acquired from https://git.esss.dk/wedel/wfm_stitching

Additionally, to other package dependencies are `fabio` and `astropy`: `conda install fabio astropy`

In [2]:
# created by Owen Arnold (optional e-mail) on 2018-09-19
# last modified by Peter M. Kadletz (peter.kadletz@esss.se) on 2018-11-12
import fabio
import numpy as np
import glob
import sys
import csv
import os

import scipp as sc
import numpy as np
from scipp import Dim

from mantid.simpleapi import *
from mantid.api import AlgorithmManager

from wfm_stitching import WFMProcessor

# defining grouping of 2D detector pixels
grouping_number = 12
nx_target = grouping_number
ny_target = grouping_number

# Number of acquired pulses per measurement
pulse_nr_reference = 1.0 / 770956
pulse_nr_sample = 1.0 / 1280381
pulse_nr_sample_elastic = 1.0 / 2416839
pulse_nr_plastic = 1.0 / 2614343

# let's get the process started:
experiment_dir = "/home/dtasev/dev/scipp_imaging/RAL_Mantid_Sep2018/"

tofs_path = os.path.join(experiment_dir, 'data_GP2', 'metadata', 'GP2_Stress_time_values.txt')
sample_path = os.path.join(experiment_dir, 'data_GP2', 'Stress Experiments', '2) R825')
sample_elastic_path = os.path.join(experiment_dir, 'data_GP2', 'Stress Experiments', '3) R825 600 Mpa')
ob_path = os.path.join(experiment_dir, 'data_GP2', 'Stress Experiments', '1) R825 Open Beam')

# These use the range notation of [start, stop, step]
# for R825
frame_parameters = [(15450,22942,64),
                    (24800,32052,64),
                    (33791,40084,64),
                    (41763,47457,64),
                    (49315,54500,64),
                    (56500,58360,64)]

# for BBC
# frame_parameters = [(16e3, 23e3, 64),
#                     (24.5e3, 32.5e3, 64),
#                     (33.5e3, 40.2e3, 64),
#                     (41.7e3, 47.73e3, 64),
#                     (49.29e3, 55.11e3, 64),
#                     (56.57e3, 60e3, 64)]

pulse_nr_reference = 1.0 / 770956
pulse_nr_sample = 1.0 / 1280381
pulse_nr_sample_elastic = 1.0 / 2416839
pulse_nr_plastic = 1.0 / 2614343

# Two options availabe for frame shift increments: 1) Robins Calibrated Values 2) Owens Calculated Values.

frame_shift_increments = [-6630.0, -2420.0, -2253.0, -2095.0, -1946.0, -1810.0]
# frame_shift_increments = [-6630,-2423,-2252,-2058,-1949,-1576]
frame_shifts = [sum(frame_shift_increments[:i + 1]) for i in range(len(frame_shift_increments))]
# rebin region reduced to cut off frames that contain no data
rebin_parameters = (8500, 43000, 64)
# instrument_filename = os.path.join(experiment_dir, 'IDF', 'V20_Definition_GP2.xml')
# accumulated_pulses_sample = 225000
# accumulated_pulses_reference = 250000
# scaling_factor = float(accumulated_pulses_reference) / float(accumulated_pulses_sample)

In [3]:
# Some helper functions

def read_x_values(tof_file):
    """
    Reads the TOF values from the CSV into a list
    """
    tof_values = list()
    with open(tof_file) as fh:
        csv_reader = csv.reader(fh, delimiter='\t')
        next(csv_reader, None)  # skip header
        for row in csv_reader:
            tof_values.append(float(row[1]))
    return tof_values


def _load_tiffs(tiff_dir):
    if not os.path.isdir(tiff_dir):
        raise RuntimeError(tiff_dir + " is not directory")
    stack = []
    path_length = len(tiff_dir) + 1
    filenames = sorted(glob.glob(tiff_dir + "/*.tiff"))
    nfiles = len(filenames)
    count = 0
    print(f"Loading {nfiles} files from '{tiff_dir}'")
    for filename in filenames:
        count += 1
        print('\r{0}: Image {1}, of {2}'.format(filename[path_length:], count, nfiles), end="")
        img = fabio.open(os.path.join(tiff_dir, filename))
        stack.append(np.flipud(img.data))

    return np.array(stack)

def tiffs_to_variable(tiff_dir):
    """
    Loads all tiff images from the directory into a scipp Variable.
    """
    stack = _load_tiffs(tiff_dir)
    return sc.Variable([Dim.Tof, Dim.Spectrum], 
                       values=stack.astype(np.float64).reshape(
                           stack.shape[0], stack.shape[1]*stack.shape[2]))

from functools import reduce

def stitch(data_array, frame_parameters, rebin_parameters):
    """
    Stitches the 5 different frames data.
    
    It crops out each frame, then shifts it so that all frames align,
    and then rebins to the common bins used for all frames.
    """
    frames = []

    rebin_params = sc.Variable([Dim.Tof], values=np.arange(*rebin_parameters, dtype=np.float64))
    
    for i, (slice_bins, shift_parameter) in enumerate(zip(frame_parameters, frame_shifts)):
        bins = sc.Variable([Dim.Tof], values=np.arange(*slice_bins, dtype=np.float64))
        # Rebins the whole data to crop it to frame bins
        rebinned = sc.rebin(data_array, Dim.Tof, bins)
        # Shift the frame backwards to make all frames overlap
        rebinned.coords[Dim.Tof] += shift_parameter
        # Rebin to overarching coordinates so that the frame coordinates align
        rebinned = sc.rebin(rebinned, Dim.Tof, rebin_params)

        frames.append(rebinned)

    for f in frames[1:]:
        frames[0] += f

    return frames[0]


def remove_special_values(values):
    np.nan_to_num(values, copy=False)
    values[values == sys.float_info.max] = 0

In [10]:
# def stitch111(data_array, frame_parameters, rebin_parameters):
#     """
#     Stitches the 5 different frames data.
    
#     It crops out each frame, then shifts it so that all frames align,
#     and then rebins to the common bins used for all frames.
#     """
#     frames = []

#     rebin_params = sc.Variable([Dim.Tof], values=np.arange(*rebin_parameters, dtype=np.float64))

#     # The bin range will be the from start of the first frame to the end of the end of the last one
#     # using the same step as the frame parameters
#     outbins = np.arange(frame_parameters[0][0], frame_parameters[-1][1], frame_parameters[0][2], 
#                                                                 dtype=np.float64)
#     out = sc.DataArray(sc.Variable([Dim.Tof, Dim.Spectrum], shape=[outbins.shape[0], data_array.shape[1]]),
#                       coords={
#                           Dim.Tof: sc.Variable([Dim.Tof], 
#                                                values=outbins, 
#                                                unit=data_array.coords[Dim.Tof].unit)})
    
#     last = 0
#     for i, (slice_bins, shift_parameter) in enumerate(zip(frame_parameters, frame_shifts)):
#         print(slice_bins)
#         start_bin_idx = np.where(rebin_params.values > slice_bins[0])[0][0]
#         end_bin_idx = np.where(rebin_params.values > slice_bins[1])[0][0]
#         print(start_bin_idx, end_bin_idx)
#         # Rebin to overarching coordinates so that the frame coordinates align
        
#         begin = last
#         end = (slice_bins[1] - slice_bins[0]) // slice_bins[2] # indices from width of the frame
#         last = end

#         # TODO figure out why 
#         # Shift the X coord and set them for the frame
#         out.coords[Dim.Tof, begin:end] = data_array[Dim.Tof, start_bin_idx:end_bin_idx].coords[Dim.Tof] + shift_parameter * sc.units.us
#         # Slice the data for this frame
#         out[Dim.Tof, begin:end] = data_array[Dim.Tof, start_bin_idx:end_bin_idx]

#     # TODO sum & maybe rebin
#     return frames[0]

# stitch111(ds["reference"], frame_parameters, rebin_parameters)

(15450, 22942, 64)
109 226


TypeError: __setitem__(): incompatible function arguments. The following argument types are supported:
    1. (self: scipp._scipp.core.CoordsProxy, arg0: scipp._scipp.core.Dim, arg1: scipp::core::VariableConstProxy) -> None

Invoked with: <scipp._scipp.core.CoordsProxy object at 0x7f1d904989d0>, (Dim.Tof, slice(0, 117, None)),     <scipp.Variable>          float64    [μs]            (Dim.Tof)  [-81.533467, -21.593407, ..., 6871.513490, 6931.453550]


In [5]:
# Structure the data (Tof, Row, Column)
ds = sc.Dataset()

ds.coords[Dim.Tof] = sc.Variable([Dim.Tof], unit=sc.units.us, values=read_x_values(tofs_path))
%time ds.coords[Dim.Tof] *= 1e3

CPU times: user 173 µs, sys: 89 µs, total: 262 µs
Wall time: 178 µs


In [6]:
# LABEL: Scaling Data

# Load the images into the dataset
# And scale each one
ds["reference"] = tiffs_to_variable(ob_path)
%time ds["reference"] *= pulse_nr_reference

Loading 1001 files from '/home/dtasev/dev/scipp_imaging/RAL_Mantid_Sep2018/data_GP2/Stress Experiments/1) R825 Open Beam'


GP2_034_Open_Beam0499.tiff: Image 500, of 1001

GP2_034_Open_Beam0760.tiff: Image 761, of 1001

GP2_034_Open_Beam1000.tiff: Image 1001, of 1001

CPU times: user 380 ms, sys: 284 ms, total: 664 ms
Wall time: 664 ms


In [ ]:
ds["sample"] = tiffs_to_variable(sample_path)
%time ds["sample"] *= pulse_nr_sample

In [ ]:
ds["sample_elastic"] = tiffs_to_variable(sample_elastic_path)
%time ds["sample_elastic"] *= pulse_nr_sample_elastic

ds.coords[Dim.Spectrum] = sc.Variable([Dim.Spectrum], values=np.arange(ds["sample"].shape[1]))
ds

In [ ]:
stitched = sc.Dataset(coords={Dim.Tof: sc.Variable([Dim.Tof], values=np.arange(*rebin_parameters, dtype=np.float64))})

# LABEL: Stitching

%time stitched["sample"] = stitch(ds["sample"], frame_parameters, rebin_parameters)

In [ ]:
stitched["reference"] = stitch(ds["reference"], frame_parameters, rebin_parameters)

In [ ]:
%time stitched["sample_elastic"] = stitch(ds["sample_elastic"], frame_parameters, rebin_parameters)

In [ ]:
# LABEL: Normalization
%time stitched["normalized"] = stitched["sample"] / stitched["reference"]

%time remove_special_values(stitched["normalized"].values)
np.max(stitched["normalized"].values)

In [ ]:
stitched["normalized_elastic"] = stitched["sample_elastic"] / stitched["reference"]
remove_special_values(stitched["normalized_elastic"].values)
np.max(stitched["normalized_elastic"].values)

In [ ]:
# LABEL: Summing spectra
%time stitched["normalized_summed"] = sc.sum(stitched["normalized"], Dim.Spectrum)

In [ ]:
stitched["normalized_elastic_summed"] = sc.sum(stitched["normalized_elastic"], Dim.Spectrum)

In [ ]:
grouping_number = 12
nx_target = grouping_number
ny_target = grouping_number

def make_map_file(nx_original, ny_original, nx_target, ny_target):
    element_width_x = nx_original // nx_target
    element_width_y = ny_original // ny_target
    
    # To contain our new spectra mappings
    grid = np.zeros((nx_original, ny_original), dtype=np.float64)

    for i in range(0, nx_target):
        for j in range(0, ny_target):
            x_start = i * element_width_x
            x_end = (i + 1) * element_width_x
            
            y_start = j * element_width_y
            y_end = (j + 1) * element_width_y

            vals = np.full((element_width_x, element_width_y), i + j * nx_target, dtype=np.float64)
            grid[x_start:x_end, y_start:y_end] = vals
    
    return sc.Variable([Dim.Spectrum], values=grid.flatten())


stitched.labels["detector_mapping"] = make_map_file(324, 324, nx_target, ny_target)
stitched.labels["detector_mapping"]

In [ ]:
# LABEL: Group detectors
dm1d = sc.groupby(stitched["normalized"], "detector_mapping", Dim.Row)
%time stitched["normalized_grpd"] = dm1d.sum(Dim.Spectrum)

In [ ]:
def get_pos(pos):
    return [pos.X(), pos.Y(), pos.Z()]

def make_component_info(ws):
    sourcePos = ws.componentInfo().sourcePosition()
    samplePos = ws.componentInfo().samplePosition()

    def as_var(pos):
        return sc.Variable(value=np.array(get_pos(pos)),
                           dtype=sc.dtype.vector_3_float64,
                           unit=sc.units.m)

    return as_var(sourcePos), as_var(samplePos)

def init_pos(ws):
    nHist = ws.getNumberHistograms()
    pos = np.zeros([nHist, 3])

    spec_info = ws.spectrumInfo()
    for i in range(nHist):
        if spec_info.hasDetectors(i):
            p = spec_info.position(i)
            pos[i, :] = [p.X(), p.Y(), p.Z()]
        else:
            pos[i, :] = [np.nan, np.nan, np.nan]
    return sc.Variable([sc.Dim.Spectrum],
                       values=pos,
                       unit=sc.units.m,
                       dtype=sc.dtype.vector_3_float64)

def load_component_info_from_instrument_file(ds, file):
    try:
        import mantid.simpleapi as mantid
        from mantid.api import Workspace
    except ImportError:
        raise ImportError(
            "Mantid Python API was not found, please install Mantid framework "
            "as detailed in the installation instructions (https://scipp."
            "readthedocs.io/en/latest/getting-started/installation.html)")
    ws = mantid.Load(file)
    
    source_pos, sample_pos = make_component_info(ws)
    
    ds.labels["source_position"] = source_pos
    ds.labels["sample_position"] = sample_pos
    ds.labels["position"] = init_pos(ws)


In [ ]:
load_component_info_from_instrument_file(stitched, '/home/dtasev/dev/scipp_imaging/RAL_Mantid_Sep2018/IDF/V20_Definition_GP2.xml')
stitched

In [ ]:
# makes the position a DataConstProxy otherwise groupby won't take it
stitched["position"] = stitched.labels["position"]
dm1d = sc.groupby(stitched["position"], "detector_mapping", Dim.Row)
position = dm1d.mean(Dim.Spectrum)

# can't do stitched.labels["position"] = position because Labels won't take a DataArray
# also can't do stitched.labels["position"] = sc.Variable(position) because then sc.convert 
# thinks stitched.labels["position"] is dimensionless (as it's actually still a DataArray)

pos = np.zeros((position.shape[0], 3))
for i, val in enumerate(position.values):
    pos[i, :] = val

# finally add it back!
stitched.labels["position"] = sc.Variable(position.dims, pos, unit=sc.units.m, dtype=position.dtype)

In [ ]:
# LABEL: Convert Units
%time stitched["normalized_wl"] = sc.neutron.convert(stitched["normalized_grpd"], Dim.Tof, Dim.Wavelength)